In [1]:
%load_ext watermark
%watermark

2017-10-22T19:33:22+02:00

CPython 3.6.1
IPython 5.3.0

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.10.0-37-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


Vamos a usar el dataset del [Titanic](https://www.kaggle.com/c/titanic/data), que contiene los datos de los pasajeros del Titanic, famoso barco que se hundió en su primer viaje en 1912. Dicho dataset contiene los datos de los pasajeros así como información sobre si sobrevivieron o no.

**Descripción de los datos**

* superviviente 	 El pasajero sobrevivió 	0 = No, 1 = Si

* clase_billete 	 Clase de camarote 	1 = Primera clase, 2 = Segunda, 3 = Tercera clase (la más pobre)

* genero 	         hombre/mujer 	

* edad 	

* n_hermanos_esposos Número de hermanos o pareja a bordo del Titanic

* n_hijos_padres     Número de hijos o padres a bordo del Titanic 

* precio_billete 	

* puerto_salida      Puerto donde el pasajero tomó el barco (C=Cherbourg, Q=Queenstown, S=Southampton)


In [1]:
import pandas as pd

In [2]:
datos = pd.read_csv("data/titanic.csv")

In [3]:
datos.head()

,superviviente,clase_billete,genero,edad,n_hermanos_esposos,n_hijos_padres,precio_billete,puerto_salida
0,0,3,hombre,22.0,1,0,7.2500,S
1,1,1,mujer,38.0,1,0,71.2833,C
2,1,3,mujer,26.0,0,0,7.9250,S
3,1,1,mujer,35.0,1,0,53.1000,S
4,0,3,hombre,35.0,0,0,8.0500,S


Los algoritmos de creación de árboles están en el submódulo de `sklearn.tree` 

En cuanto al tipo de algoritmo para crear árboles, scikit-learn [usa una versión optimizada del algoritmo CART](http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart) *(Classification and Regression Trees)*, que  permite usar árboles de decisión tanto para problemas de clasificación como de regresión.

In [4]:
from sklearn import tree
from sklearn.model_selection import cross_val_score

In [5]:
arbol = tree.DecisionTreeClassifier()

In [6]:
from sklearn import preprocessing

In [7]:
columnas_categoricas = ["genero", "puerto_salida"]

In [8]:
codificador_binario = preprocessing.LabelBinarizer()

In [9]:
datos_categoricos = pd.get_dummies(datos[columnas_categoricas])

In [10]:
datos_categoricos

,genero_hombre,genero_mujer,puerto_salida_C,puerto_salida_Q,puerto_salida_S
0,1,0,0,0,1
1,0,1,1,0,0
2,0,1,0,0,1
3,0,1,0,0,1
4,1,0,0,0,1
5,1,0,0,1,0
6,1,0,0,0,1
7,1,0,0,0,1
8,0,1,0,0,1
9,0,1,1,0,0


In [11]:
pasajeros = (
    pd.concat([
        datos.drop(columnas_categoricas, axis=1),
        datos_categoricos
    ],axis=1
    )
)
pasajeros.edad = pasajeros.edad.fillna(pasajeros.edad.mean())

In [12]:
pasajeros.head()

,superviviente,clase_billete,edad,n_hermanos_esposos,n_hijos_padres,precio_billete,genero_hombre,genero_mujer,puerto_salida_C,puerto_salida_Q,puerto_salida_S
0,0,3,22.0,1,0,7.2500,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,0,1,1,0,0
2,1,3,26.0,0,0,7.9250,0,1,0,0,1
3,1,1,35.0,1,0,53.1000,0,1,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,0,0,1


In [13]:
arbol.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [29]:
cross_val_score(arbol, pasajeros.drop("superviviente", axis=1), pasajeros.superviviente, scoring="roc_auc", 
                cv=10).mean()

0.76873032849503442

Una funcionalidad interesante que tienen los arboles de decision en `sklearn` es que se pueden visualizar con `Graphviz`. Para ello tenemos que instalar la libreria con `pip install graphviz`.

In [30]:
%matplotlib inline

In [31]:
!pip install graphviz

In [32]:
import graphviz

def dibujar_arbol(arbol):
    dot_data = tree.export_graphviz(arbol, out_file=None, 
                         feature_names=pasajeros.drop("superviviente", axis=1).columns,  
                         filled=True, 
                         impurity=False,
                         rounded=True,  
                         special_characters=True)  
    
    graph = graphviz.Source(dot_data)
    graph.format = 'png'
    graph.render('arbol',view=True)

In [33]:
dibujar_arbol(arbol)

Alternativamente, se puede exportar el árbol y abrirlo posteriormente con graphviz desde la terminal (o desde la página http://webgraphviz.com/ que renderiza archivos de graphviz)

In [34]:
tree.export_graphviz(arbol, out_file="arbol.dot")

In [35]:
cat arbol.dot

digraph Tree {
node [shape=box] ;
0 [label="X[6] <= 0.5\ngini = 0.473\nsamples = 891\nvalue = [549, 342]"] ;
1 [label="X[1] <= 6.5\ngini = 0.306\nsamples = 577\nvalue = [468, 109]"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="X[2] <= 2.5\ngini = 0.444\nsamples = 24\nvalue = [8, 16]"] ;
1 -> 2 ;
3 [label="gini = 0.0\nsamples = 15\nvalue = [0, 15]"] ;
2 -> 3 ;
4 [label="X[3] <= 1.5\ngini = 0.198\nsamples = 9\nvalue = [8, 1]"] ;
2 -> 4 ;
5 [label="gini = 0.0\nsamples = 5\nvalue = [5, 0]"] ;
4 -> 5 ;
6 [label="X[1] <= 3.5\ngini = 0.375\nsamples = 4\nvalue = [3, 1]"] ;
4 -> 6 ;
7 [label="X[4] <= 39.144\ngini = 0.5\nsamples = 2\nvalue = [1, 1]"] ;
6 -> 7 ;
8 [label="gini = 0.0\nsamples = 1\nvalue = [0, 1]"] ;
7 -> 8 ;
9 [label="gini = 0.0\nsamples = 1\nvalue = [1, 0]"] ;
7 -> 9 ;
10 [label="gini = 0.0\nsamples = 2\nvalue = [2, 0]"] ;
6 -> 10 ;
11 [label="X[0] <= 1.5\ngini = 0.28\nsamples = 553\nvalue = [460, 93]"] ;
1 -> 11 ;
12 [label="X[4] <= 26.144\ngini = 0.

Otra funcionalidad que tienen los árboles de decisión en sklearn es que nos dan una indicación de la importancia de cada variable en el modelo, almacenada en el atributo `feature_importances_`. Calcula la importancia en función de la ganancia de información de cada variable, es decir, que variables separan mejor las distintas clases.

In [36]:
arbol.feature_importances_

array([ 0.10751562,  0.25475341,  0.05215079,  0.02532001,  0.23113876,
        0.        ,  0.30933519,  0.00669015,  0.        ,  0.01309608])

In [37]:
dict(zip(
    pasajeros.drop("superviviente", axis=1),
    arbol.feature_importances_
))

{'clase_billete': 0.10751561696823284,
 'edad': 0.25475340530869411,
 'genero_hombre': 0.0,
 'genero_mujer': 0.30933518862833864,
 'n_hermanos_esposos': 0.052150787399534285,
 'n_hijos_padres': 0.025320013763985374,
 'precio_billete': 0.23113875781128776,
 'puerto_salida_C': 0.0066901461674494156,
 'puerto_salida_Q': 0.0,
 'puerto_salida_S': 0.013096083952477564}

Por ejemplo, en este árbol vemos que las variables que tienen más peso a la hora de decidir si un pasajero sobrevive o no son la edad, el género y el precio del billete (¡las mujeres y los niños primero!).

In [38]:
tree.DecisionTreeClassifier?

Éstos son los parámetros más importantes para los modelos DecisionTreeClassifier de sklearn:

* criterion : El criterio para calcular la reducción de impureza (ganancia de información) al hacer una partición. Se puede elegir entre `gini`, o `entropy` 

* max_depth : La profundidad máxima del árbol. Definimos profundidad como el número de nodos que atraviesa una observación *(cuantas "preguntas" se le hacen)*.

* max_features:  El máximo numero de particiones potenciales se consideran al evaluar un nodo.

* max_leaf_nodes : Límite de hojas para el árbol.

* min_impurity_decrease : la ganancia de información mínima en un nodo para hacer una partición. (Si no hay ninguna partición que cumpla este criterio, se para el desarrollo del árbol en dicho nodo).

* class_weight : Para clases imbalanceadas, podemos pasar el argumento class_weight, como un diccionario de `{clase: peso}` para que sklearn tenga en cuenta los pesos. Alternativamente, podemos pasar el string `balanced` para que sklearn genere pesos en función del número de muestras de cada clase.

Los árboles de decisión tienden a sobreajustar cuando tienen demasiada complejidad. Los siguientes parámetros ayudan a controlarla:

* min_samples_leaf : Mínimo número de observaciones en un nodo para considerar un nodo como una hoja. Por defecto es 1, ésto significa que sklearn solo para de hacer particiones cuando un nodo tiene sólo 1 observación.

* min_samples_split : Mínimo número de observaciones en un nodo para generar una partición. Por defecto es 2, esto significa que sklearn por defecto siempre va a intentar particionar un nodo con 2+ elementos.

In [39]:
arbol_simple = tree.DecisionTreeClassifier(max_depth=3)

In [40]:
arbol_simple.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [41]:
dibujar_arbol(arbol_simple)

In [43]:
cross_val_score(arbol_simple, pasajeros.drop("superviviente", axis=1), 
                pasajeros.superviviente, scoring="roc_auc", cv=10).mean()

0.85065461336049586

In [44]:
arbol_balanceado = tree.DecisionTreeClassifier(max_depth=3, class_weight="balanced")

In [45]:
arbol_balanceado.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [49]:
dibujar_arbol(arbol_balanceado)

In [55]:
cross_val_score(arbol_balanceado, pasajeros.drop("superviviente", axis=1), 
                pasajeros.superviviente, scoring="roc_auc", cv=10).mean()

0.86121264465382108

In [56]:
cross_val_score(arbol_balanceado, pasajeros.drop("superviviente", axis=1), pasajeros.superviviente,
                scoring="precision", cv=10).mean()

0.74139176772198745

Además del algoritmo CART para generar árboles, scikit-learn también proporciona una clase de arboles llamada ExtraTreeClassifier, o Extremely Random Trees (Árboles Extremadamente Aleatorios). En estos árboles, en lugar de seleccionar en cada nodo la párticion que proporciona la mayor ganancia de información, ¡se decide una partición al azar!.

In [50]:
arbol_aleatorio = tree.ExtraTreeClassifier(max_features=1)

In [51]:
arbol_aleatorio.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

dibujar_arbol(arbol_aleatorio)

Los árboles aleatorios funcionan peor que los árboles de decisión por sí mismos, pero usados en grupo pueden funcionar mejor.

In [59]:
cross_val_score(arbol_aleatorio, pasajeros.drop("superviviente", axis=1), pasajeros.superviviente, scoring="roc_auc",
                cv=10).mean()

0.74176561129502305

Los árboles aleatorios funcionan peor que los árboles de decisión por sí mismos, pero usados en grupo pueden funcionar mejor.